In [ ]:
def master_array(gen_array, sim_array, cut_array):
    
    id_particle = ak.to_numpy(gen_array["event"])
    id_reco = ak.to_numpy(sim_array["event"])
    id_both = ak.to_numpy(cut_array["event"])
    print("Start masking arrays")
    # Mask out overlapping events
    mask_particle_only = ~np.isin(id_particle, id_both)
    mask_reco_only = ~np.isin(id_reco, id_both)

    particle_only = gen_array[mask_particle_only]
    reco_only = sim_array[mask_reco_only]
    
    particle_only = ak.with_field(particle_only, True, "pass_particle")
    particle_only = ak.with_field(particle_only, False, "pass_reco")

    reco_only = ak.with_field(reco_only, False, "pass_particle")
    reco_only = ak.with_field(reco_only, True, "pass_reco")

    cut_array = ak.with_field(cut_array, True, "pass_particle")
    cut_array = ak.with_field(cut_array, True, "pass_reco")
    
    print("Done masking") 
    
    reco_field_order = [
    'run', 'event', 'eventWeight',
    'ptl1', 'truth_ptl1',
    'etal1', 'truth_etal1',
    'phil1', 'truth_phil1',
    'ptb1', 'truth_ptb1', 'ptb2', 'truth_ptb2',
    'ptb3', 'truth_ptb3', 'ptb4', 'truth_ptb4',
    'etab1', 'truth_etab1', 'etab2', 'truth_etab2',
    'etab3', 'truth_etab3', 'etab4', 'truth_etab4',
    'phib1', 'truth_phib1', 'phib2', 'truth_phib2',
    'phib3', 'truth_phib3', 'phib4', 'truth_phib4',
    'mb1', 'truth_mb1', 'mb2', 'truth_mb2',
    'mb3', 'truth_mb3', 'mb4', 'truth_mb4',
    'ptj1', 'truth_ptj1', 'ptj2', 'truth_ptj2',
    'ptj3', 'truth_ptj3', 'ptj4', 'truth_ptj4',
    'ptj5', 'truth_ptj5', 'ptj6', 'truth_ptj6',
    'etaj1', 'truth_etaj1', 'etaj2', 'truth_etaj2',
    'etaj3', 'truth_etaj3', 'etaj4', 'truth_etaj4',
    'etaj5', 'truth_etaj5', 'etaj6', 'truth_etaj6',
    'phij1', 'truth_phij1', 'phij2', 'truth_phij2',
    'phij3', 'truth_phij3', 'phij4', 'truth_phij4',
    'phij5', 'truth_phij5', 'phij6', 'truth_phij6',
    'mj1', 'truth_mj1', 'mj2', 'truth_mj2',
    'mj3', 'truth_mj3', 'mj4', 'truth_mj4',
    'mj5', 'truth_mj5', 'mj6', 'truth_mj6',
    'met', 'truth_met', 'metphi', 'truth_metphi',
    'ptwhad', 'truth_ptwhad', 'truth_pt_particlewhad',
    'rapiditywhad', 'truth_rapidity_particlewhad',
    'mwhad', 'truth_mwhad',
    'mbl_selected', 'truth_m_particlebl',
    'mbwhad_selected', 'truth_m_particlebwhad',
    'dRbl_selected', 'truth_dR_particlebl',
    'dRbwhad_selected', 'truth_dR_particlebwhad',
    'pass_particle', 'pass_reco'
    ]

    # Fill missing fields with None and keep only listed fields in specified order
    def harmonize_array(array, target_fields):
        for field in target_fields:
            if field not in array.fields:
                array = ak.with_field(array, None, field)
        return ak.zip({field: array[field] for field in target_fields})
    print("Start harmonizing fields")
    # Apply harmonization
    truth_array = harmonize_array(particle_only, reco_field_order)
    cut_array = harmonize_array(cut_array, reco_field_order)
    print("Done harmonizing")
    df_particle = ak.to_dataframe(particle_only)
    df_reco = ak.to_dataframe(reco_only)
    df_matched = ak.to_dataframe(cut_array)
    print("Converted to pandas - Start concatenating")
    # Concatenate
    df_final = pd.concat([df_particle, df_reco, df_matched], ignore_index=True)

    #final_array = ak.concatenate([particle_only, reco_only, cut_array], axis=0)
    
    return df_final

In [ ]:
def avoid_double_counting(array):
    ids = array["event"]

    # Use numpy to find unique ones and return indices of the first occurrences
    _, unique_index = np.unique(ak.to_numpy(ids), return_index=True)

    # Sort indices to preserve original order
    unique_index = np.sort(unique_index)

    # Select only unique entries
    unique_awkward = array[unique_index]
    return unique_awkward

#truth_array = avoid_double_counting(truth_array)
#herwig_array = avoid_double_counting(herwig_array)
#pythia_array_reco_cuts = avoid_double_counting(pythia_array_reco_cuts)
#pythia_array_matched_cuts = avoid_double_counting(pythia_array_matched_cuts)